In [119]:
op=open("D:\WebDevelopment\langchain\openAPIKEY.txt","r")
f = open("D:\WebDevelopment\langchain\HuggingFaceAPI.txt", "r")
gs=open("D:\WebDevelopment\langchain\googlesearchAPIkey.txt","r")
g_cse=open("D:\WebDevelopment\langchain\google_cse_id.txt")
API_KEY=f.readline()
Google_searchKey=gs.readline()
Google_CSE=g_cse.readline()
OpenAPI_KEY=op.readline()
f.close()
gs.close()
g_cse.close()
op.close()


In [120]:
import os
os.environ["GOOGLE_CSE_ID"] = Google_CSE
os.environ["GOOGLE_API_KEY"] = Google_searchKey
os.environ['OPENAI_API_KEY']=OpenAPI_KEY

In [121]:
from typing import List
from pydantic import BaseModel,Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [122]:
class Tagging(BaseModel):
    """Tagging piece of the text with particular info."""
    sentiment:str=Field(description="sentiment of text, should be`pos`,`neg`, or `neutral`")
    language:str=Field(description="language of text (should be ISO 639-1 code)")

In [123]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tagging piece of the text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be`pos`,`neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [124]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
from langchain_experimental.chat_models import Llama2Chat

In [125]:
model = ChatOpenAI(temperature=0)

In [133]:

'''llm = HuggingFaceEndpoint(
    repo_id="TheBloke/Llama-2-7B-Chat-GGML",
    #task="text-classification",
    temperature=0.1,
    max_new_tokens=20,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=API_KEY,
)'''

'llm = HuggingFaceEndpoint(\n    repo_id="TheBloke/Llama-2-7B-Chat-GGML",\n    #task="text-classification",\n    temperature=0.1,\n    max_new_tokens=20,\n    do_sample=False,\n    repetition_penalty=1.03,\n    huggingfacehub_api_token=API_KEY,\n)'

In [127]:
#model=Llama2Chat(llm=llm)

In [128]:
tagging_functions=[convert_pydantic_to_openai_function(Tagging)]

In [129]:
prompt=ChatPromptTemplate.from_messages([
    ("system","Think carefully, and then tag the text as instructed"),
    ("user","{input}")
])

In [130]:
model_with_functions=model.bind(
    functions=tagging_functions,
    function_call={"name":"Tagging"}
)

In [131]:
tagging_chain=prompt|model_with_functions

In [132]:
tagging_chain.invoke({"input":"I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f0d783c0-5092-430b-97cd-e859c99e29bf-0')

In [135]:
tagging_chain.invoke({"input":"non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5b53aaa-c64e-4336-8769-ef7336f9308f-0')

In [136]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [137]:
tagging_chain=prompt|model_with_functions|JsonOutputFunctionsParser()

In [138]:
tagging_chain.invoke({"input":"non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

Extraction

In [147]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [151]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [152]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [153]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1347e578-6a01-4941-bd00-2dcaac93c1fa-0')

In [155]:
prompt=ChatPromptTemplate.from_messages([
    ("system","Extract the relevant information, if not explicitly provide do not guess. Extract partical info."),
    ("human","{input}")
])

In [156]:
extraction_chain=prompt|extraction_model

In [157]:
extraction_chain.invoke({"input":"joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 114, 'total_tokens': 136}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ac4e477c-7217-467f-ae0d-bc1a9fb5b555-0')

In [160]:
extraction_chain=prompt|extraction_model|JsonOutputFunctionsParser()

In [161]:
extraction_chain.invoke({"input":"joe is 30, his mom is Martha"})

{'people': [{'name': 'joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [162]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [165]:
extraction_chain=prompt|extraction_model|JsonKeyOutputFunctionsParser(key_name="people")

In [166]:
extraction_chain.invoke({"input":"joe is 30, his mom is Martha"})

[{'name': 'joe', 'age': 30}, {'name': 'Martha', 'age': None}]

Doing it for real
We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [169]:
from langchain.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents=loader.load()

In [170]:
doc=documents[0]
page_content=doc.page_content[:10000]
print(page_content)







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [171]:
class Overview(BaseModel):
    """overview of a section of text."""
    summary:str=Field(description="Provide a concise summary of the content")
    language:str=Field(description="Provide the language of the content written in.")
    keywords:str=Field(description="provide the keywords related to the content.")

In [176]:
overview_tagging_function=[
    convert_pydantic_to_openai_function(Overview)
]
tagging_model=model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain=prompt|tagging_model|JsonOutputFunctionsParser()

In [177]:
tagging_chain.invoke({"input":page_content})

{'summary': 'This article discusses the concept of building autonomous agents powered by LLM (large language model) as the core controller. It covers components such as planning, memory, and tool use, along with challenges and references.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, challenges, references'}

In [178]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    tilte:str
    author:Optional[str]

In [179]:
class Info(BaseModel):
    """Information to extract"""
    papers:List[Paper]

In [180]:
paper_extraction_function=[
    convert_pydantic_to_openai_function(Info)
]
extraction_model=model.bind(
    functions=paper_extraction_function,
    function_call={"name":"Info"}
)
extraction_chain=prompt|extraction_model|JsonOutputFunctionsParser()

In [183]:
extraction_chain.invoke({"input":page_content})

{'papers': [{'tilte': 'LLM Powered Autonomous Agents',
   'author': 'Lilian Weng'}]}

In [185]:
templete="""A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt=ChatPromptTemplate.from_messages([
    ("system",templete),
    ("human","{input}")
])

In [186]:
extraction_chain=prompt|extraction_model|JsonKeyOutputFunctionsParser(key_name="papers")

In [188]:
extraction_chain.invoke({"input":page_content})

[{'tilte': 'Chain of thought (CoT; Wei et al. 2022)', 'author': None},
 {'tilte': 'Tree of Thoughts (Yao et al. 2023)', 'author': None},
 {'tilte': 'LLM+P (Liu et al. 2023)', 'author': None},
 {'tilte': 'ReAct (Yao et al. 2023)', 'author': None},
 {'tilte': 'Reflexion (Shinn & Labash 2023)', 'author': None},
 {'tilte': 'Chain of Hindsight (CoH; Liu et al. 2023)', 'author': None},
 {'tilte': 'Algorithm Distillation (AD; Laskin et al. 2023)', 'author': None}]

In [189]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splliter=RecursiveCharacterTextSplitter(chunk_overlap=0)

In [190]:
splits=text_splliter.split_text(doc.page_content)

In [191]:
len(splits)

15

In [208]:
def flatten(matrix):
    flat_list=[]
    for row in matrix:
        flat_list+=row
    return flat_list

#flatten([[1,2], [3,4]])

In [209]:
from langchain.schema.runnable import RunnableLambda

In [210]:
prep=RunnableLambda(
    lambda x:[{"input":doc} for doc in text_splliter.split_text(x)]
)

In [211]:
prep.invoke("hi")

[{'input': 'hi'}]

In [214]:
chain=prep|extraction_chain.map()|flatten

In [215]:
chain.invoke(doc.page_content)

[{'tilte': 'AutoGPT', 'author': None},
 {'tilte': 'GPT-Engineer', 'author': None},
 {'tilte': 'BabyAGI', 'author': None},
 {'tilte': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'tilte': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'tilte': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'tilte': 'ReAct', 'author': 'Yao et al. 2023'},
 {'tilte': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'tilte': 'Chain of Hindsight', 'author': 'Liu et al. 2023'},
 {'tilte': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'tilte': 'Laskin et al. 2023', 'author': None},
 {'tilte': 'Miller 1956', 'author': None},
 {'tilte': 'Duan et al. 2017', 'author': None},
 {'tilte': 'Google Blog', 'author': None},
 {'tilte': 'MRKL (Karpas et al. 2022)', 'author': None},
 {'tilte': 'TALM (Tool Augmented Language Models; Parisi et al. 2022)',
  'author': None},
 {'tilte': 'Toolformer (Schick et al. 2023)', 'author': None},
 {'tilte': 'HuggingGPT (Shen et al. 2023)', 'author': None},
 {'tilte'